## Introdução
Este notebook prepara o texto do teor das decisões jurídicas extraídas do e-Saj com o webscrapper. Do diretório raiz do projeto é possível acessar o diretório _dados_. Dentro dele, o diretório _raw_data_ contém arquivos csv com código do processo, classe, assunto, magistrado, comarca, foro, vara, data da disponibilização no e-Saj e teor de 500 decisões de segunda instância. Uma vez que existem muitos arquivos csv, é preciso percorrê-los, um a um, aplicando o tratamento textual.

As informações estruturadas supracitadas se repetem no teor dos documentos, então é necessário separar o conteúdo textual do teor destas informações, o que é feito com uma expressão regular. Em seguida, é preciso percorrer todos os arquivos csv de _raw_data_ e aplicar a remoção de _stopwords_, colcoar todo o conteúdo em caixa baixa, remover acentuação e substituir os números por uma expressão genérica, _NUM_. Os arquivos tratados substituem os originais. Uma vez que os arquivos tratados contém o teor original, a substituição não implica em perda de dados originais.



In [1]:
%load_ext autoreload
%autoreload 2

import string
import unidecode
import re
import nltk
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
from tqdm import tqdm

expr_teor = re.compile('(\\n\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\n\\n\\n.*TRIBUNAL DE JUSTI.*V[iI][sS][tT][oO][sS]|\\n\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\n\\n\\nTRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\\n\\n.*Juiz.* de Direito:.+?\\n)', re.S)
tqdm.pandas(desc="barra de progresso")
stemmer = nltk.stem.RSLPStemmer()

def tratar_textos(raw_corpus):
    raw_corpus = raw_corpus.str.replace('[\r\n\]',' ')
    pt_stopwords = set([unidecode.unidecode(i) for i in nltk.corpus.stopwords.words('portuguese')])  
    def preprocess(x, *stopwords_list):
        x = unidecode.unidecode(x.lower())
        x = [stemmer.stem(w) for w in x.split() if w not in set(pt_stopwords)]
        return ' '.join(x)
    
    
    clean_text = raw_corpus.apply(preprocess, args = pt_stopwords)
    clean_text = clean_text.str.replace('\d+', '_NUM_')
    return clean_text

def preparar_teores(df, expr):
    lista = df['Teor'].apply(lambda x: True if expr.search(str(x)) else False)
    df = df[lista]
    textos_raw = df['Teor'].apply(lambda x: re.sub(expr, '', str(x)))
    df['Teor_preprocessado'] = tratar_textos(textos_raw)
    return df

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


#### Extração do teor prepocessado

In [35]:
directory = "dados/raw_data/"
for filename in tqdm(os.listdir(directory)):
    try:      
        df = pd.read_csv(os.path.join(directory, filename), index_col=0)
        lista = df['Teor'].apply(lambda x: True if expr_teor.search(str(x)) else False)
        df = df[lista]
        textos_raw = df['Teor'].apply(lambda x: re.sub(expr_teor, '', str(x)))
        df_prep = preparar_teores(df, expr_teor)
        df_prep.to_csv(os.path.join(directory, filename))
    except:
            e = sys.exc_info()[0]
            print(e)

100%|██████████| 434/434 [3:47:34<00:00, 31.46s/it]  


In [2]:
dados = pd.read_csv('dados/raw_data/27_10_19_65500_66000.csv')
dados.head()

,Unnamed: 0,id,Classe,Assunto,Magistrado,Comarca,Foro,Vara,Data_disp,Teor,Teor_preprocessado,T
0,0.0,0002965-34.2017.8.26.0008,Cumprimento de sentença,Assunto não informado.,Rubens Pedreiro Lopes,SÃO PAULO,Foro Regional VIII - Tatuapé,4ª Vara Cível,17/10/2019,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\n\n\n\nTRIBUNAL...,". o(a) executado(a) satisfez obrigacao, confor...",execut satisf obrigaca conform inform s fl _NU...
1,1.0,0044702-71.2013.8.26.0100,Usucapião,Usucapião Ordinária,ALINE APARECIDA DE MIRANDA,SÃO PAULO,Foro Central Cível,1ª Vara de Registros Públicos,17/10/2019,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\n\nTRIBUNAL DE ...,. trata-se acao usucapiao promovida ruth tomaz...,trat aca usucapia promov ruth tomaz oliv cruz ...
2,2.0,0014992-06.2013.8.26.0100,Usucapião,Usucapião Ordinária,Vivian Labruna Catapani,SÃO PAULO,Foro Central Cível,1ª Vara de Registros Públicos,17/10/2019,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\n\nTRIBUNAL DE ...,". aparecida pereira garbo pascoal garbo, quali...",aparec per garb pasco garb qualific aut ajuiz ...
3,3.0,1000367-38.2019.8.26.0132,Embargos de Terceiro Cível,Constrição / Penhora / Avaliação / Indisponibi...,MARIO YAMADA FILHO,Catanduva,Foro de Catanduva,Vara de Família e Sucessões,17/10/2019,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\n\nTRIBUNAL DE ...,". trata-se ""embargos terceiro"" opostos juliana...","trat ""embarg terceiro"" opost juli cristin mari..."
4,4.0,1001806-65.2019.8.26.0009,Despejo por Falta de Pagamento Cumulado Com Co...,Inadimplemento,Cristiane Sampaio Alves Mascari Bonilha,SÃO PAULO,Foro Regional IX - Vila Prudente,3ª Vara Cível,17/10/2019,\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n\n\nTRIBUNAL DE ...,. _NUM_. trata-se acao despejo falta pagamento...,_NUM_ trat aca despej falt pag cumul cobranc p...
